In [2]:
import json
import pandas as pd
import re

In [3]:
df = pd.read_csv('data/final_extraction/final_extraction.csv')
df

,id,year,title,json_output
0,0,2024,Role of carrier delocalization in enhancing th...,"{'perovskite_composition': 'MAPbI3', 'electron..."
1,1,2024,Perovskite-Based Tandem Solar Cells,"{'perovskite_composition': None, 'electron_tra..."
2,2,2024,Efficiency assessment of perovskite solar cell...,"{'perovskite_composition': None, 'electron_tra..."
3,3,2024,Multifaceted Design of Surface Passivator for ...,{'perovskite_composition': 'Cs 0.05 FA 0.9 MA ...
4,4,2024,An Universal Multi-Additive Strategy to Enhanc...,"{\r\n ""perovskite_composition"": ""Cs 0.05 MA..."
...,...,...,...,...
2534,2534,2023,Supporting Information In-situ Vanadium Modifi...,"{'perovskite_composition': 'Cu2ZnSn(S,Se)4 (CZ..."
2535,2535,2024,Nanosecond Laser Passivation Mechanism of Q235...,"{'perovskite_composition': None, 'electron_tra..."
2536,2536,2024,Improve Perovskite Solar Cell Stability and Ef...,"{'perovskite_composition': None, 'electron_tra..."
2537,2537,2024,Electron-hole diffusion lengths > 175 m mm in ...,"{'perovskite_composition': 'CH3NH3PbI3', 'elec..."


In [12]:
def escape_internal_quotes(json_string):
    # This regex finds values within quotes and fixes internal unescaped quotes
    return re.sub(r'":\s*"([^"]*?)"', lambda m: '": "' + m.group(1).replace('"', '\\"') + '"', json_string)

In [13]:
def force_fix(json_string):
    '''
    {"perovskite_composition": "Cs0.05(FA0.98MA0.02)0.95Pb(I0.98Br0.02)3", "electron_transport_layer": "6,6"-phenyl-C61-butyric acid methyl ester", "pin_nip_structure": "NIP", "hole_transport_layer": "2-(3,6-dimethoxy-9H-carbazol-9-yl)ethyl phosphonic acid", "test_1": {"test_name": "ISOS-L", "temperature": "25", "time": "1000", "humidity": "50-60", "passivating_molecule": "β-poly(1,1-difluoroethylene)", "control_pce": "22.3", "treated_pce": "24.6", "control_voc": "1.13", "treated_voc": "1.18"}}
    '''
    ## convert {" pattern to ZS
    json_conv = re.sub(r'{\"',"ZS", json_string)
    ## convert ": to ZT
    json_conv = re.sub(r'\":', "ZT", json_conv)
    ## Convert _" to ZP
    json_conv = re.sub(r' \"', "ZP", json_conv)
    ## Convert ", to ZV
    json_conv = re.sub(r'\",Z', "ZVZ", json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub(r'\"}', "ZQ", json_conv)

    ##Perform real conversion interested
    json_conv = json_conv.replace('"', "'")

    #Revert Everything 

    ## convert ZS to {" 
    json_conv = re.sub("ZS", r'{"', json_conv)
    ## convert ": to ZT
    json_conv = re.sub("ZT", r'":', json_conv)
    ## Convert _" to ZP
    json_conv = re.sub("ZP", r' "', json_conv)
    ## Convert ", to ZV
    json_conv = re.sub("ZV", r'",', json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub("ZQ", r'"}', json_conv)

    return json_conv

In [55]:
def clean_json(json_string):
    if "```json" in json_string:
        json_string = json_string.split("```json")[1]
        json_match = re.search(r"\{.*\}", json_string, re.DOTALL)
        if json_match:
            json_string = json_match.group(0).strip()
    json_string = json_string.replace("None", "null")
    json_string = json_string.replace("'", "\"")
    json_string = escape_internal_quotes(json_string)
    json_string = force_fix(json_string)
    json_string = re.sub(r'(".*?")', lambda m: m.group(1).replace(":", ""), json_string)
    return json_string

In [ ]:
import re
output = {}
to_format = {}
i = 0
for index, row in df.iterrows():
    i += 1
    json_match = re.search(r"\{.*\}", row["json_output"], re.DOTALL)
    if json_match:
        raw_json = json_match.group(0).strip()
    else:
        raw_json = None
    try:
        raw_json = json.loads(clean_json(raw_json))
    except json.JSONDecodeError as e:
        print(row["id"])
        to_format[row["id"]] =clean_json(raw_json)
        
    output[str(row["id"])] = raw_json

In [ ]:
with open('data/final_extraction/to_format.json', 'w') as f:
    json.dump(to_format, f)

In [24]:
with open('data/final_extraction/to_format_formatted.json', 'r') as f:
    formatted = json.load(f)
    

In [26]:
output.update(formatted)

In [33]:
output = {k: v for k, v in output.items() if "test" not in k}

In [35]:
with open('data/final_extraction/final_extraction_formatted.json', 'w') as f:
    json.dump(output, f)

In [46]:
with open('data/final_extraction/final_extraction_formatted.json', 'r') as f:
    data = json.load(f)

In [60]:
to_format = {}
for key in data:
    if type(data[key]) == dict:
        continue
    try:
        json.loads(clean_json(data[key]))
        print(key)
    except json.JSONDecodeError as e:
        to_format[key] = clean_json(data[key])

In [62]:
with open('data/final_extraction/to_format.json', 'w') as f:
    json.dump(to_format, f)

In [64]:
import pandas as pd
df = pd.read_json('data/final_extraction/final_extraction_formatted.json')
data = df.T.sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2538 entries, 0 to 2538
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   perovskite_composition    1036 non-null   object
 1   electron_transport_layer  958 non-null    object
 2   pin_nip_structure         772 non-null    object
 3   hole_transport_layer      873 non-null    object
 4   passivating_molecule      2051 non-null   object
 5   control_pce               1229 non-null   object
 6   control_voc               996 non-null    object
 7   treated_pce               1592 non-null   object
 8   treated_voc               1159 non-null   object
 9   test_1                    2160 non-null   object
 10  test_2                    636 non-null    object
 11  structure_pin_nip         55 non-null     object
 12  treated_jsc               68 non-null     object
 13  treated_ff                93 non-null     object
 14  test_3                    210

In [65]:
data[(data["passivating_molecule"].isna() == False) & (data["control_pce"].isna() == False) & (data["treated_pce"].isna() == False)]

,perovskite_composition,electron_transport_layer,pin_nip_structure,hole_transport_layer,passivating_molecule,control_pce,control_voc,treated_pce,treated_voc,test_1,...,treated_eqe,treated_stabilized_pce,control_fill_factor,treated_fill_factor,controljsc,treatedjsc,treated_hysteresis_index,binding_energy,fill_factor,treated_iscc
0,MAPbI3,PEDOT:PSS,NIP,PCBM,DAAB,4.5,0.87,12.6,0.92,"{'test_name': 'ISOS-L', 'temperature': 30, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cs 0.05 FA 0.9 MA 0.05 PbI 3,None,None,None,Cyanoguanidine diiodide,20.44,1077,23.04,1119,"{'test_name': 'ISOS-D', 'temperature': None, '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg),TiO2,NaN,Spiro-OMeTAD,tBBAI,14.91,1.07,23.5,1.14,"{'stability_type': 'ISOS-L-1', 'temperature': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FAPbI3 0.85 (-MAPbBr3) 0.15,Spiro-OMeTAD,NIP,None,7-amino-4-trifluoromethylcoumarin (C151),20.44,1.09,22.46,1.14,"{'test_name': 'aging', 'temperature': 20, 'tim...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,None,None,None,ICBA,22.4,1.15,25.1,1.19,"{'test_name': None, 'temperature': 40, 'time':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,CsPbI3,TiO2,n-i-p,spiro-OMeTAD,n-octylammonium iodide (n-OAI),18.6,1.18,19.8,1.23,"{'test_name': 'ISOS-L', 'temperature': 25, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2530,None,None,None,None,TDMASn,15,None,18.2,None,"{'test_name': 'Stability test', 'temperature':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2532,MAPbI3,None,None,None,OTTM,18.45,1.08,21.01,1.12,"{'test_name': 'stability', 'temperature': 85, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,MAPbI3,PCBM,PIN,NiOx,PbS,15.26,1.002,18.74,1.025,"{'test_name': 'ISOS-T', 'temperature': 400, 't...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
data

,perovskite_composition,electron_transport_layer,pin_nip_structure,hole_transport_layer,passivating_molecule,control_pce,control_voc,treated_pce,treated_voc,test_1,...,treated_eqe,treated_stabilized_pce,control_fill_factor,treated_fill_factor,controljsc,treatedjsc,treated_hysteresis_index,binding_energy,fill_factor,treated_iscc
0,MAPbI3,PEDOT:PSS,NIP,PCBM,DAAB,4.5,0.87,12.6,0.92,"{'test_name': 'ISOS-L', 'temperature': 30, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,None,None,formamidine sulfinic acid (FSA),None,None,27,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,None,None,None,None,None,None,None,"{'test_name': 'ISOS-D', 'temperature': 85, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cs 0.05 FA 0.9 MA 0.05 PbI 3,None,None,None,Cyanoguanidine diiodide,20.44,1077,23.04,1119,"{'test_name': 'ISOS-D', 'temperature': None, '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg),TiO2,NaN,Spiro-OMeTAD,tBBAI,14.91,1.07,23.5,1.14,"{'stability_type': 'ISOS-L-1', 'temperature': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534,"Cu2ZnSn(S,Se)4 (C:SSe)",None,NaN,None,Vanadium (V),8.28,0.454,11.11,0.458,"{'stability_type': None, 'temperature': None, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535,None,None,None,None,None,None,None,None,None,"{'test_name': None, 'temperature': 25, 'time':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2536,None,TiO2,None,Spiro-MeOTAD,K-NAA,19.86,1.14,None,None,"{'test_name': 'ISOS-D', 'temperature': 25, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2537,CH3NH3PbI3,None,None,"Au, Ga",None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
has_passivator = data[data["passivating_molecule"].str.contains("perovskite_composition") == False]

In [74]:
has_passivator.to_csv('data/final_extraction/df_has_passivator.csv')

In [75]:
list(has_passivator["passivating_molecule"])

['DAAB',
 'formamidine sulfinic acid (FSA)',
 'Cyanoguanidine diiodide',
 'tBBAI',
 '7-amino-4-trifluoromethylcoumarin (C151)',
 '[BMP][BF4]',
 'Thioacetamide (TAA)',
 'ICBA',
 'MASCN and FASCN',
 'DPDC',
 'Chen et al. introduced Rb2CO3 layer at the SnO2/perovskite interface',
 'N-Ti 3 C 2',
 'PAA',
 'MeO-2PACz',
 'SPM/PEAI',
 '2-Mercaptopyrimidine',
 'Imidazole',
 'N2200',
 'BDAI2',
 '4-chlorobenzylammonium chloride (75%) and 4-chlorobenzylammonium bromide (25%)',
 'HI',
 'poly(2-(2-methoxyethoxy)ethylmethacrylate)',
 'DFH',
 'S-benzyl-L-cysteine',
 'DTPA-CN',
 'Molybdenum-sulfur-containing precursor',
 'Urea',
 'Oxygen (O2)',
 '4-fluorobenzamine trifluoroacetate',
 'Cs0.05MA0.15FA0.8Pb(I1-xBr_x)_3',
 'Oleylammonium chloride (OAmCl)',
 '2-thiopheneacetic acid (TPA)',
 'Siloxane-based self-assembled molecule with nitrile group',
 'Phenethylammonium iodide (PEAI)',
 'Potassium laurate (KLA)',
 'KCFSO',
 '(5-AVA)x(MA)1-x PbI3',
 'TPPP(Cl)',
 '1-iodobutane, 1-phenylethylamine, PbI2',
 '2-